In [70]:
import re
import string
import pandas as pd
import numpy as np
from collections import Counter
import sqlite3
import json
import csv

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# import fasttext
# import gensim
# from gensim.models import Word2Vec
# from gensim.models import ldaseqmodel
# from gensim import corpora
# import gensim.downloader as api

from lxml import etree

import matplotlib.pyplot as plt


from time import time
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

plt.rcParams['figure.dpi'] = 300
plt.rcParams["font.family"] = "serif"


def clean_text(text, tokenizer, stopwords):
    """Pre-process text and generate tokens

    Args:
        text: Text to tokenize.

    Returns:
        Tokenized text.
    """
    text = str(text).lower()  # Lowercase words
    text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  # Remove punctuation

    tokens = tokenizer(text)  # Get tokens from text
    tokens = [t for t in tokens if not t in stopwords]  # Remove stopwords
    tokens = ["" if t.isdigit() else t for t in tokens]  # Remove digits
    tokens = [t for t in tokens if len(t) > 1]  # Remove short tokens
    
    # stemming and lem
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    
    return tokens

In [71]:
# Read in xmls
# select sections containing discussion
tree = etree.parse('./papers/conversationalAgent/HC_paper_all/xml/17.xml')

# select sections containing discussion
nodes = tree.xpath("(//*[("
        "contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz')"
        ",'rq1:') or "
        "contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz')"
        ",'rq 1:')"
        ")]) [last()]")
if len(nodes) != 0:
    node = nodes[0]
    title = ''.join(node.xpath("text()"))
    text = ''.join(node.xpath("../descendant::*/text()")[1:])
    # print(title)
else:
    print("NOT FOUND!")
title

"Although all these works rely on rapport-building conversational strategies, few of them investigate how rapport-building dialogues influence the perceived quality of the items recommended, or people's compliance towards the recommendations. Moreover, they do not investigate the impact of users' interaction mode on users' perceptions. In this paper, we aim at building a conversational recommender system that recommends recipes while building rapport with its users. More specifically, in this paper, we focus on the following research questions: RQ1: How does the way users interact with a conversational recommender system influence their perception of and their intention to cook recommended recipes? RQ2: How do a conversational recommender system's conversational strategies influence users' perception of and their intention to cook recommended recipes?"

In [72]:
def get_introduction_text(path):
    """Get discussion text from xml file.
    
    Args:
        path: Paper path.
    
    Returns:
        Discussion text.
    """
    tree = etree.parse(path)
    nodes = tree.xpath("(//*[local-name()='head' and ("
            "contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz')"
            ",'introduction')"
            ")]) [last()]")
    if len(nodes) != 0:
        node = nodes[0]
        title = ''.join(node.xpath("text()"))
        text = ''.join(node.xpath("../descendant::*/text()")[1:])
        # print(title)
    else:
        print("NOT FOUND!")
        text = ''
    return text


def get_discussion_text(path):
    """Get discussion text from xml file.
    
    Args:
        path: Paper path.
    
    Returns:
        Discussion text.
    """
    tree = etree.parse(path)
    nodes = tree.xpath("(//*[local-name()='head' and ("
            "contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz')"
            ",'discussion') or "
            "contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz')"
            ",'implication')"
            ")]) [last()]")
    if len(nodes) != 0:
        node = nodes[0]
        title = ''.join(node.xpath("text()"))
        text = ''.join(node.xpath("../descendant::*/text()")[1:])
        # print(title)
    else:
        print("NOT FOUND!")
        text = ''
    return text


def get_relatedWork_text(path):
    """Get discussion text from xml file.
    
    Args:
        path: Paper path.
    
    Returns:
        related work text.
    """
    tree = etree.parse(path)
    nodes = tree.xpath("(//*[local-name()='head' and ("
            "contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz')"
            ",'related work') or "
            "contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz')"
            ",'background')"
            ")]) [last()]")
    if len(nodes) != 0:
        node = nodes[0]
        title = ''.join(node.xpath("text()"))
        text = ''.join(node.xpath("../descendant::*/text()")[1:])
        # print(title)
    else:
        print("NOT FOUND!")
        text = ''
    return text

def get_RQ_text(path):
    # Read in xmls
    # select sections containing discussion
    tree = etree.parse(path)

    # select sections containing discussion
    nodes = tree.xpath("(//*[("
            "contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz')"
            ",'rq1:') or "
            "contains(translate(text(), 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz')"
            ",'rq 1:')"
            ")]) [last()]")
    if len(nodes) != 0:
        node = nodes[0]
        title = ''.join(node.xpath("text()"))
        text = ''.join(node.xpath("../descendant::*/text()")[1:])
        # print(title)
    else:
        print("NOT FOUND!")
    return title

In [73]:
path = r'./papers/conversationalAgent/HC_paper_all/xml/17.xml'

In [74]:
get_introduction_text(path)

'Healthy eating implies complex decision making processes [6], including being aware of healthy options and choosing among them [24]. One solution to overcome this issue and help people to make healthier choices is to develop health-aware food recommender systems [31]. While significant effort has been put recently into optimizing the food selection algorithms [30], many other factors can also influence users\' overall experience when interacting with a recommender system [14]. Indeed, the way the recommendation is presented [18], the system\'s response time [33], or even the length of the system\'s utterances [20] can have an influence on users\' perception of the system.One trend to improve users\' experience is to make the interaction more natural by designing the recommendation process as a conversation [23]. Besides helping users to achieve task-oriented goals, conversations can also fulfill interpersonal functions, such as building rapport [29]. Rapport can be described as a dyna

In [75]:
get_RQ_text(path)

"Although all these works rely on rapport-building conversational strategies, few of them investigate how rapport-building dialogues influence the perceived quality of the items recommended, or people's compliance towards the recommendations. Moreover, they do not investigate the impact of users' interaction mode on users' perceptions. In this paper, we aim at building a conversational recommender system that recommends recipes while building rapport with its users. More specifically, in this paper, we focus on the following research questions: RQ1: How does the way users interact with a conversational recommender system influence their perception of and their intention to cook recommended recipes? RQ2: How do a conversational recommender system's conversational strategies influence users' perception of and their intention to cook recommended recipes?"

In [76]:
get_relatedWork_text(path)

"Food Recommender Systems. A common approach for food recommender systems is to recommend a recipe based on its ingredients. In [8], for example, the authors developed a system that relies on recipes that people like to infer their preferred ingredients. The system then recommends new recipes containing the previously inferred ingredients. In [9], the authors developed a system that collects users' preferences by asking them to rate and tag the recipes they usually cook at home. The system then relies on user's preferences to rank recipes and deliver recommendations with the highest scores. This Matrix Factorization algorithm outperformed the content-based approach proposed by [8]. Other approaches only rely on dietary information to recommend recipes that would match users' needs. YumMe, the recommender system developed in [36], automatically extracts dietary information from pictures of recipes to form a user profile. The system then relies on this user profile to deliver subsequent 